In [1]:
import pandas as pd 
import numpy as np
import glob
import re

In [2]:
def bitmask_flag_change(Data_Frame, colname):
    """ Creates a list of values that flag the change in validity of the Eyetracking data, 
    diferentiates between the beggining and enf of an invalid event. 
    
Parameters
-------------
     
     :dataframe(DataFrame): your data frame,
     :colname(str): Name of the column that containts the validity mask 
returns 
----------

    List of len DataFrame with values that mark changes on the validity column
     """
    change_flag = ["No_change"] # initialize flag 
    for i in range(1, len(Data_Frame)):
        if Data_Frame[colname][i] == Data_Frame[colname][i-1]: # compare each value with the previous value
          change_flag.append("No_change") 
        elif Data_Frame[colname][i] != Data_Frame[colname][i-1]:
            if Data_Frame[colname][i] == 3:
                  change_flag.append("Invalid_Section_Ends")
            elif Data_Frame[colname][i] == 0:
                change_flag.append("Invalid_Section_Starts")
            else:
                change_flag.append("ERROR")
    return change_flag

In [3]:
# Since the collider names are too detailed, here we create a dictionary with patterns to classify them into our categories of interest
patterns = {'\d{2}_Sa':'Passive_Agent', '\d{2}_Cma':'Active_Agent', 'Building_\d+': 'Building'}
patterns.update(dict.fromkeys(['Castle-TaskBuilding_56','HighSilo-TaskBuilding_49', 'Windmill-TaskBuilding_10_1', 'Church_TaskBuilding_16'], 'Global_Landmark'))
patterns.update(dict.fromkeys(['TaskBuilding_2','TaskBuilding_3', 'TaskBuilding_5', 'TaskBuilding_8', 'TaskBuilding_9', 'TaskBuilding_11', 'TaskBuilding_13', 'TaskBuilding_14', 'TaskBuilding_20', 'TaskBuilding_21', 'TaskBuilding_23','TaskBuilding_27', 'TaskBuilding_29', 'TaskBuilding_32', 'TaskBuilding_34',  'TaskBuilding_38', 'TaskBuilding_41', 'TaskBuilding_42', 'TaskBuilding_44', 'TaskBuilding_45', 'TaskBuilding_47', 'TaskBuilding_50', 'TaskBuilding_51', 'TaskBuilding_52', 'BasketballCourt_58', 'Construction_57', 'Graffity_02', 'Graffity_03', 'Graffity_05', 'Graffity_08', 'Graffity_09', 'Graffity_11', 'Graffity_13', 'Graffity_14', 'Graffity_20', 'Graffity_21', 'Graffity_23', 'Graffity_27', 'Graffity_29', 'Graffity_32', 'Graffity_34', 'Graffity_38', 'Graffity_41', 'Graffity_42', 'Graffity_44', 'Graffity_45', 'Graffity_47',  'Graffity_50', 'Graffity_51', 'Graffity_52'], 'TaskBuilding_Public'))
patterns.update(dict.fromkeys(['TaskBuilding_1','TaskBuilding_4', 'TaskBuilding_6', 'TaskBuilding_7', 'TaskBuilding_12', 'TaskBuilding_15', 'TaskBuilding_17', 'TaskBuilding_18', 'TaskBuilding_19', 'TaskBuilding_22', 'TaskBuilding_24','TaskBuilding_25', 'TaskBuilding_26', 'TaskBuilding_28', 'TaskBuilding_30',  'TaskBuilding_31', 'TaskBuilding_33', 'TaskBuilding_35', 'TaskBuilding_36', 'TaskBuilding_37', 'TaskBuilding_39', 'TaskBuilding_40', 'TaskBuilding_43', 'TaskBuilding_48', 'TaskBuilding_54','TaskBuilding_55','Graffity_1','Graffity_4', 'Graffity_6', 'Graffity_7', 'Graffity_12', 'Graffity_15', 'Graffity_17', 'Graffity_18', 'Graffity_19', 'Graffity_22', 'Graffity_24','Graffity_25', 'Graffity_26', 'Graffity_28', 'Graffity_30',  'Graffity_31', 'Graffity_33', 'Graffity_35', 'Graffity_36', 'Graffity_37', 'Graffity_39', 'Graffity_40', 'Graffity_43', 'Graffity_48', 'Graffity_54', 'Graffity_55' ], 'TaskBuilding_Residential'))
default_val = 'Background'

In [4]:
path = "/Volumes/SSD/00_Data_Processing/Pre_processed/03_Individuals_IndividualSessions" 

  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
    # content.append(filename)
    One_participant = pd.read_csv(filename)
    One_participant.drop(['Unnamed: 0',"level_0", "index"],axis=1, inplace=True)
    #Apply function that marks beggining and end of invalid sections
    One_participant["Bitmask_flag"] = bitmask_flag_change(One_participant, "combinedGazeValidityBitmask")
    One_participant.reset_index(inplace=True)
    # Get index of begging and end of invalid events
    indexLastValid = One_participant[One_participant["Bitmask_flag"] == "Invalid_Section_Starts"].index
    indexLastInValid = One_participant[One_participant["Bitmask_flag"] == "Invalid_Section_Ends"].index
    # Get 20 rows prior to the beggining of the invalid event 
    indexLower = indexLastValid - 20
    indexUpper = indexLastValid
    # Since it's possible that the invalid event occured less than 20 rows from the beggining of the file
    # we need to correct so that the highest possible index is the firs value in the file
    indexLower_r = [0 if i < 0 else i for i in indexLower]
    #Create a list with the most common element 200ms before the invalid section started
    Elements_to_replace = [One_participant.iloc[down:up,20].mode().iloc[0] for down, up in zip(indexLower_r, indexUpper)]
    #Create new column for interpolated events
    One_participant["Interpolated_collider"] = One_participant["hitObjectColliderName"]
    #Concat the row indexes that need replacement
    ranges = list(zip(indexLastValid, indexLastInValid))
    #Replace the invalid event with the mode of 20 events prior
    for i, (lower, upper) in enumerate(ranges):
        One_participant.iloc[lower:upper,-2]  = Elements_to_replace[i]
    # Here we look for the patterns contained in the dictionary and create the more general/informative variable Collider_Categorical
    One_participant['Collider_Categorical'] = One_participant['Interpolated_collider'].apply(lambda x: next((val for key, val in patterns.items() if re.match(key, x)), default_val))
    One_participant['Collider_Categorical']
    # Replace coordinates and eucledian distances with Nans
    One_participant.loc[One_participant['combinedGazeValidityBitmask'] == 0, ['hitPointOnObject_x', 'hitPointOnObject_y', 'hitPointOnObject_z', 'Eucledian_distance']] =  np.nan 
    #Saves an individual file per session per subject with out duplicates
    One_participant.to_csv(f"/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated/{filename[-10:-4]}.csv", index=True)
    print(filename[-10:-4])
    indexLastValid = []
    indexLastInValid = []
    indexLower = []
    indexUpper = []
    content.append(One_participant)

# converting content to data frame
data_frame = pd.concat(content)

0479_2
0479_3
0479_5
1754_1
1754_2
1754_3
1754_4
1754_5
2258_1
2258_2
2258_3
2258_4
2258_5
2361_2
2361_3
2361_4
2361_5
2693_1
2693_2
2693_3
2693_4
2693_5
3246_1
3246_2
3246_3
3246_5
3310_1
3310_2
3310_3
3310_4
3310_5
3572_1
3572_2
3572_3
3572_4
3976_1
3976_2
3976_3
3976_4
3976_5
4176_1
4176_2
4176_3
4176_4
4176_5
4796_1
4796_2
4796_3
4796_4
4796_5
4917_1
4917_2
4917_3
4917_4
4917_5
5238_1
5238_2
5238_3
5238_4
5531_1
5531_3
5531_4
5531_5
5741_1
5741_2
5741_3
5741_4
5741_5
6642_1
6642_2
6642_3
6642_4
6642_5
7093_1
7093_2
7093_3
7093_4
7093_5
7264_1
7264_2
7264_3
7264_4
7264_5
7412_1
7412_2
7412_3
7412_4
7412_5
7842_1
7842_2
7842_3
7842_4
7842_5
8007_1
8007_2
8007_3
8007_4
8007_5
8469_1
8469_2
8469_3
8469_4
8469_5
8673_1
8673_2
8673_3
8673_4
8673_5
8695_2
8695_3
8695_4
8695_5
9472_1
9472_2
9472_3
9472_4
9472_5
9502_1
9502_2
9502_3
9502_5
9601_1
9601_2
9601_3
9601_4
9601_5
0479_1
